In [1]:
#Impostar librerias
import pyodbc
import pandas as pd
import os
import mysql.connector
from pathlib import Path

In [2]:
# Configurar la conexión a MySQL
conn = mysql.connector.connect(
    host="localhost",       # Servidor de MySQL
    user="root",      # Nombre de usuario
    password="Aerosmith@cdc96",  # Contraseña
    database="KayData"  # Nombre de la base de datos
)
# Crear un cursor para ejecutar consultas
cursor = conn.cursor()
# Ruta de la carpeta que contiene los archivos de Excel
carpeta_excel = Path('C:/Users/chris/OneDrive/Documentos/Diagnóstivo SPG/Formularios de evaluación')

In [3]:
def insertar_productor(df, cursor, conn):
    # Extraer datos (ajusta los índices según tu archivo)
    nombres_veedor = df.iloc[4, 4]  # Ajusta los índices
    fecha_levantamiento = df.iloc[5, 4]
    hora_inicio = df.iloc[5, 6]
    hora_finalizacion = df.iloc[5, 8]
    nombre_productor = df.iloc[6, 3]
    estado_civil = df.iloc[7, 1]
    sexo = df.iloc[7, 4]
    edad = df.iloc[7, 7]
    nivel_educacion = df.iloc[8, 2]
    organizacion = df.iloc[8, 6]
    telefono = df.iloc[9, 1]
    ubicacion = df.iloc[9, 6]

    # Verificar duplicados
    query_verificacion = """
    SELECT COUNT(*) 
    FROM productores 
    WHERE nombre_productor = %s 
      AND fecha_levantamiento = %s 
      AND hora_inicio = %s;
    """
    valores_verificacion = (nombre_productor, fecha_levantamiento, hora_inicio)
    cursor.execute(query_verificacion, valores_verificacion)
    resultado = cursor.fetchone()[0]

    if resultado > 0:
        print(f"El registro ya existe en la base de datos.")
        return False
    else:
        # Insertar datos
        query_insercion = """
        INSERT INTO Productores (
            nombres_veedor, fecha_levantamiento, hora_inicio, hora_finalizacion,
            nombre_productor, estado_civil, sexo, edad, nivel_educacion, organizacion, telefono, ubicacion
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        valores_insercion = (
            nombres_veedor, fecha_levantamiento, hora_inicio, hora_finalizacion,
            nombre_productor, estado_civil, sexo, edad, nivel_educacion, organizacion, telefono, ubicacion
        )
        cursor.execute(query_insercion, valores_insercion)
        conn.commit()
        print("Datos del productor insertados correctamente.")
        return True

In [4]:
def insertar_resultados(df, productor_id, cursor, conn):
    # Iterar sobre las filas del DataFrame
    for index, row in df.iterrows():
        # Extraer datos de la fila
        dimension = row["Dimensión"] if pd.notna(row["Dimensión"]) else "N/A"
        subdimension = row["Subdimensión"] if pd.notna(row["Subdimensión"]) else "N/A"
        seccion = row["Sección"] if pd.notna(row["Sección"]) else "N/A"
        pregunta = row["Pregunta"] if pd.notna(row["Pregunta"]) else "N/A"
        resultado = row["Resultado"] if pd.notna(row["Resultado"]) else "N/A"
        puntuacion = float(row["Puntuación"]) if pd.notna(row["Puntuación"]) else 0.0

        # Insertar datos en Resultados
        query_insercion = """
        INSERT INTO Resultados (
            productor_id, dimension, subdimension, seccion, pregunta, resultado, puntuacion
        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        valores_insercion = (
            productor_id, dimension, subdimension, seccion, pregunta, resultado, puntuacion
        )
        cursor.execute(query_insercion, valores_insercion)
        conn.commit()
        print(f"Datos de resultados insertados correctamente: {pregunta}")

In [5]:
for archivo in carpeta_excel.glob('*.xlsx'):
    print(f"Procesando archivo: {archivo}")

    # Leer la hoja FEvaluación (datos del productor)
    df_productor = pd.read_excel(archivo, sheet_name='FEvaluación', header=None)

    # Insertar datos en Productores
    if insertar_productor(df_productor, cursor, conn):
        # Obtener el ID del productor recién insertado
        cursor.execute("SELECT LAST_INSERT_ID()")
        productor_id = cursor.fetchone()[0]

        # Leer la hoja REvaluación (datos de las dimensiones)
        df_resultados = pd.read_excel(archivo, sheet_name='REvaluación2')

        # Insertar datos en Resultados
        insertar_resultados(df_resultados, productor_id, cursor, conn)

# Cerrar la conexión
cursor.close()
conn.close()
print("Proceso completado.")

Procesando archivo: C:\Users\chris\OneDrive\Documentos\Diagnóstivo SPG\Formularios de evaluación\Paul_Naranjo-12-03-25.xlsx
Datos del productor insertados correctamente.
Datos de resultados insertados correctamente: ¿En qué porcentaje depende de insumos externos (Fuera de la parcela) Ej. Semillas Gallinaza, Biol; Sulfocálcico etc.?
Datos de resultados insertados correctamente: ¿En el transcurso de este año Ud. ha gestionado o recibido capacitaciones o insumos para la producción agroecológica?
Datos de resultados insertados correctamente: ¿Para Ud. que es soberanía alimentaria?
Datos de resultados insertados correctamente: ¿Qué semillas Ud. Utiliza para la producción agroecológica y en qué porcentaje?c(TRADICIONAL)
Datos de resultados insertados correctamente: ¿Qué semillas Ud. Utiliza para la producción agroecológica y en qué porcentaje?c(HIBRIDAS)
Datos de resultados insertados correctamente: ¿De las semillas tradicionales con que especies cuenta en este momento?
Datos de resultados i